In [1]:
#| default_exp wandb_manager

---
title: "Downloading from Weights and Biases"
author: "Juma Shafara"
date: "10-01-2024"
---

In [1]:
#| hide
#| export

import wandb
import pandas as pd
import os
import shutil

def loadWandbArtifact(entity: str = 'analytic', 
                      project: str = 'Risk Assessment', 
                      artifact_name: str = None, 
                      version: str = "latest", 
                      dataset:bool=True,
                      download_dir: str = "artifacts", 
                      download: bool = False) -> pd.DataFrame:
    """
    Loads a specified WandB artifact as a DataFrame if it's a dataset, 
    and optionally deletes the downloaded artifacts folder after loading.

    Parameters:
    - entity (str): Your WandB username or team name.
    - project (str): The WandB project name.
    - artifact_name (str): The name of the artifact to download.
    - version (str): The version of the artifact to download (default is 'latest').
    - download_dir (str): The local directory to save the downloaded artifact (default is 'artifacts').
    - download (bool): Whether to store a copy of the artifact on local disk

    Returns:
    - pd.DataFrame: Loaded dataset as a DataFrame if applicable.
    """
    # Initialize WandB API
    api = wandb.Api()

    # Fetch the artifact
    artifact = api.artifact(f"{entity}/{project}/{artifact_name}:{version}")

    # Download the artifact to the specified directory
    artifact_dir = artifact.download(download_dir)

    # Check if there is a CSV or similar dataset file and load it
    if dataset:
        df = None
        for file in os.listdir(artifact_dir):
            file_path = os.path.join(artifact_dir, file)
            if file.endswith('.csv'):
                df = pd.read_csv(file_path)
                break
            elif file.endswith('.xlsx'):
                df = pd.read_excel(file_path)
                break

    # Optionally delete the artifact directory after loading
    if download:
        print(file_path)
    else:
        shutil.rmtree(download_dir)

    return df


## Loading datasets to memory

To load a dataset from the weights and biases, use the `loadWandbArtifact` from `wandb_manager` as illustrated below

In [3]:
from rtvpy.wandb_manager import loadWandbArtifact

dataset = loadWandbArtifact(
    artifact_name='2021_year2_set.csv')

dataset.head(n=2)

wandb:   1 of 1 files downloaded.  


,year,hhh_age,pre_cohort,pre_hhid,pre_village,pre_cluster,pre_parish,pre_subcounty,pre_district,region,...,Material_roof_dec,Material_floor_dec,Fuel_source_cooking_dec,Cooking_tech_dec,Lighting_source_dec,Type_of_Toilet_Facility_dec,Bedding_type_adults_dec,perennial_crops_grown_dec,Business_type_dec,save_mode_dec
0,1,75,2021,KAN-KAN-FER-K1768,Kangyeyo,Rutendere,Rutendere,Kayonza,Kanungu,NaN,...,['Iron Sheets/Concrete/Tiles/Asbestos/Other'],['Earth'],['Firewood/Dung/Grass/Reeds'],['Traditional Stone open Air'],['Electricity/Solar'],['Uncovered Pit Latrine / Covered Pit Latrine ...,['Mattress'],['Food Banana'],NaN,['No savings']
1,1,32,2021,KAN-OMU-KAN-K4711,Omurwambu,Kihanda,Kihanda,Kirima,Kanungu,NaN,...,['Iron Sheets/Concrete/Tiles/Asbestos/Other'],['Cement'],['Firewood/Dung/Grass/Reeds'],['Traditional Stone open Air'],['Electricity/Solar'],['Uncovered Pit Latrine / Covered Pit Latrine ...,"['Cloth', 'Bed frame', 'Mattress']","['Food Banana', 'Alcohol Banana', 'Avocado']",NaN,['RTV VSLA (Cash round)']


## Downloading datasets

The example above does not store a copy of the dataset on your local machine, if you would like a copy stored on your local machine, you can specify the `download` parameter as showed

In [4]:
from rtvpy.wandb_manager import loadWandbArtifact

dataset = loadWandbArtifact(
    artifact_name='2021_year2_set.csv', 
    download=True)

dataset.head(n=2)

wandb:   1 of 1 files downloaded.  


artifacts/2021_year2_set.csv


,year,hhh_age,pre_cohort,pre_hhid,pre_village,pre_cluster,pre_parish,pre_subcounty,pre_district,region,...,Material_roof_dec,Material_floor_dec,Fuel_source_cooking_dec,Cooking_tech_dec,Lighting_source_dec,Type_of_Toilet_Facility_dec,Bedding_type_adults_dec,perennial_crops_grown_dec,Business_type_dec,save_mode_dec
0,1,75,2021,KAN-KAN-FER-K1768,Kangyeyo,Rutendere,Rutendere,Kayonza,Kanungu,NaN,...,['Iron Sheets/Concrete/Tiles/Asbestos/Other'],['Earth'],['Firewood/Dung/Grass/Reeds'],['Traditional Stone open Air'],['Electricity/Solar'],['Uncovered Pit Latrine / Covered Pit Latrine ...,['Mattress'],['Food Banana'],NaN,['No savings']
1,1,32,2021,KAN-OMU-KAN-K4711,Omurwambu,Kihanda,Kihanda,Kirima,Kanungu,NaN,...,['Iron Sheets/Concrete/Tiles/Asbestos/Other'],['Cement'],['Firewood/Dung/Grass/Reeds'],['Traditional Stone open Air'],['Electricity/Solar'],['Uncovered Pit Latrine / Covered Pit Latrine ...,"['Cloth', 'Bed frame', 'Mattress']","['Food Banana', 'Alcohol Banana', 'Avocado']",NaN,['RTV VSLA (Cash round)']


As you can see, the path to the dataset will be specified at the bottom like "artifacts/2021_year2_set.csv"

## (down)Load from other accounts or/and projects

The code snippets above automatically attempt to load/download the datasets from the 'Risk Assessment' project of the 'analytics' team. If you would like to access a dataset from a difference team or/and project, you can set `entity` and `project` parameters of the `loadWandbArtifact()` function as showed below

In [1]:
from rtvpy.wandb_manager import loadWandbArtifact

dataset = loadWandbArtifact(
    entity='analytic',
    project='Risk Assessment',
    artifact_name='2021_year2_set.csv', 
    download=True)

dataset.head(n=2)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb:   1 of 1 files downloaded.  


artifacts/2021_year2_set.csv


,year,hhh_age,pre_cohort,pre_hhid,pre_village,pre_cluster,pre_parish,pre_subcounty,pre_district,region,...,Material_roof_dec,Material_floor_dec,Fuel_source_cooking_dec,Cooking_tech_dec,Lighting_source_dec,Type_of_Toilet_Facility_dec,Bedding_type_adults_dec,perennial_crops_grown_dec,Business_type_dec,save_mode_dec
0,1,75,2021,KAN-KAN-FER-K1768,Kangyeyo,Rutendere,Rutendere,Kayonza,Kanungu,NaN,...,['Iron Sheets/Concrete/Tiles/Asbestos/Other'],['Earth'],['Firewood/Dung/Grass/Reeds'],['Traditional Stone open Air'],['Electricity/Solar'],['Uncovered Pit Latrine / Covered Pit Latrine ...,['Mattress'],['Food Banana'],NaN,['No savings']
1,1,32,2021,KAN-OMU-KAN-K4711,Omurwambu,Kihanda,Kihanda,Kirima,Kanungu,NaN,...,['Iron Sheets/Concrete/Tiles/Asbestos/Other'],['Cement'],['Firewood/Dung/Grass/Reeds'],['Traditional Stone open Air'],['Electricity/Solar'],['Uncovered Pit Latrine / Covered Pit Latrine ...,"['Cloth', 'Bed frame', 'Mattress']","['Food Banana', 'Alcohol Banana', 'Avocado']",NaN,['RTV VSLA (Cash round)']


## (down)Load a specific version of a dataset

Dataset may change or get updated over time, if you need to download a specific version of a dataset, you can specify as `v0`, 'v1' etc via the `version` parameter of the `loadWandbArtifact()` method as below.

In [3]:
from rtvpy.wandb_manager import loadWandbArtifact

dataset = loadWandbArtifact(
    artifact_name='2021_year2_set.csv', 
    version='v1',
    download=True)

dataset.head(n=2)

wandb:   1 of 1 files downloaded.  


artifacts/2021_year2_set.csv


,year,hhh_age,pre_cohort,pre_hhid,pre_village,pre_cluster,pre_parish,pre_subcounty,pre_district,region,...,Material_roof_dec,Material_floor_dec,Fuel_source_cooking_dec,Cooking_tech_dec,Lighting_source_dec,Type_of_Toilet_Facility_dec,Bedding_type_adults_dec,perennial_crops_grown_dec,Business_type_dec,save_mode_dec
0,1,75,2021,KAN-KAN-FER-K1768,Kangyeyo,Rutendere,Rutendere,Kayonza,Kanungu,NaN,...,['Iron Sheets/Concrete/Tiles/Asbestos/Other'],['Earth'],['Firewood/Dung/Grass/Reeds'],['Traditional Stone open Air'],['Electricity/Solar'],['Uncovered Pit Latrine / Covered Pit Latrine ...,['Mattress'],['Food Banana'],NaN,['No savings']
1,1,32,2021,KAN-OMU-KAN-K4711,Omurwambu,Kihanda,Kihanda,Kirima,Kanungu,NaN,...,['Iron Sheets/Concrete/Tiles/Asbestos/Other'],['Cement'],['Firewood/Dung/Grass/Reeds'],['Traditional Stone open Air'],['Electricity/Solar'],['Uncovered Pit Latrine / Covered Pit Latrine ...,"['Cloth', 'Bed frame', 'Mattress']","['Food Banana', 'Alcohol Banana', 'Avocado']",NaN,['RTV VSLA (Cash round)']


> you can get the latest version by setting `version='latest'`

## All parameters

Below is the list of all parameters that you can modify to get desired results

- `entity` (str): Your WandB username or team name.
- `project` (str): The WandB project name.
- `artifact_name` (str): The name of the artifact to download.
- `version` (str): The version of the artifact to download (default is 'latest').
- `download_dir` (str): The local directory to save the downloaded artifact (default is 'artifacts').
- `download` (bool): Whether to delete the downloaded artifacts folder after loading the data.


In [3]:
#| hide
import nbdev; nbdev.nbdev_export()